In [1]:
import mne
import mne_bids
import numpy as np
import pandas as pd
import autoreject as ar
import os

In [2]:
bids_root = "/Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/BIDS"
output_dir = "/Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/output"
subjects_dir = "/Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/fs_subjects"

In [12]:
for subject in ["ap", "ca", "ha", "ja", "mv", "wc", "zt"]:
    
    epochs_list = []

    for task in ["clean", "noisy"]:

        path = mne_bids.BIDSPath(root=bids_root, subject=subject, session="01", task=task, datatype="ieeg", suffix="ieeg")
        raw = mne_bids.read_raw_bids(path, extra_params=dict(preload=True))

        metadata = pd.read_csv(os.path.join(path.directory, f"{task}_metadata.csv"))

        montage = raw.get_montage()

        # add fiducials to montage
        montage.add_estimated_fiducials(f"{subjects_dir}/{subject}")

        #trans = mne_bids.get_head_mri_trans(bids_path=path, fs_subject=subject,
        #                                    fs_subjects_dir=subjects_dir)

        # now with fiducials assigned, the montage will be properly converted
        # to "head" which is what MNE requires internally (this is the coordinate
        # system with the origin between LPA and RPA whereas MNI has the origin
        # at the posterior commissure)
        raw.set_montage(montage)

        raw.filter(0.15, 200, n_jobs=-1)
        raw.notch_filter(range(60, 200, 60), method="spectrum_fit", filter_length="5s", n_jobs=-1)
        
        raw_filt.set_eeg_reference("average")
        raw_filt = raw.copy().filter(1, 100)

        events, event_id = mne.events_from_annotations(raw, event_id={"house/noisy": 11, "house/clean": 1, "face/noisy": 12, "face/clean": 2})

        epochs = mne.Epochs(raw, events, event_id, metadata=metadata, baseline=None, detrend=0, tmin=-0.2, tmax=0.8, preload=True)

        epochs_filt = mne.Epochs(raw_filt, events, event_id, baseline=None, detrend=0, tmin=-0.2, tmax=0.8, preload=True)

        reject = ar.get_rejection_threshold(epochs_filt, ch_types="ecog", verbose=True)
        print(reject)
        epochs_filt.drop_bad(reject=reject)

        ica = mne.preprocessing.ICA(n_components=0.95, max_iter="auto", method="infomax", fit_params=dict(extended=True))
        ica.fit(epochs_filt)
        epochs_clean = ica.apply(epochs.copy())

        reject = ar.get_rejection_threshold(epochs_clean, ch_types="ecog", verbose=True)
        print(reject)
        epochs_clean.drop_bad(reject=reject)

        epochs_clean.apply_baseline((None, None))

        epochs_list.append(epochs_clean)

    break
    epochs = mne.concatenate_epochs(epochs_list, add_offset=100000)
    epochs.save(f"{output_dir}/epoched/{subject}-epo.fif.gz", overwrite=True)

Extracting parameters from /Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/BIDS/sub-ap/ses-01/ieeg/sub-ap_ses-01_task-clean_ieeg.vhdr...
Setting channel info structure...
Reading 0 ... 268399  =      0.000 ...   268.399 secs...
Reading events from /Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/BIDS/sub-ap/ses-01/ieeg/sub-ap_ses-01_task-clean_events.tsv.
Reading channel info from /Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/BIDS/sub-ap/ses-01/ieeg/sub-ap_ses-01_task-clean_channels.tsv.
Reading electrode coords from /Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/BIDS/sub-ap/ses-01/ieeg/sub-ap_ses-01_space-ACPC_electrodes.tsv.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.15 - 2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband 

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    1.1s
[Parallel(n_jobs=4)]: Done  41 out of  41 | elapsed:    2.6s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    4.4s
[Parallel(n_jobs=4)]: Done  41 out of  41 | elapsed:   10.8s finished


Removed notch frequencies (Hz):
     60.00 : 4346 windows
    120.00 : 4346 windows
    180.00 : 4346 windows
ECoG channel type selected for re-referencing
Applying average reference.
Applying a custom ('ECoG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 3301 samples (3.301 sec)

Used Annotations descriptions: ['face/clean', 'house/clean']
Adding metadata with 2 columns
300 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw 

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done  41 out of  41 | elapsed:    2.3s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    6.3s
[Parallel(n_jobs=4)]: Done  41 out of  41 | elapsed:   15.6s finished


Removed notch frequencies (Hz):
     60.00 : 10865 windows
    120.00 : 10865 windows
    180.00 : 10865 windows
ECoG channel type selected for re-referencing
Applying average reference.
Applying a custom ('ECoG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 3301 samples (3.301 sec)

Used Annotations descriptions: ['face/noisy', 'house/noisy']
Adding metadata with 5 columns
630 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded R

In [10]:
import mne_icalabel

label = mne_icalabel.label_components(raw_filt, ica, method="iclabel")

/var/folders/_r/kcfrqwd143bfg9vx3jtb4l6w0000gs/T/ipykernel_507/3262485654.py:3: RuntimeWarning: The provided Raw instance does not seems to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  label = mne_icalabel.label_components(raw_filt, ica, method="iclabel")


KeyboardInterrupt: 

In [12]:
epochs_disk.event_id

{'face/clean': 2, 'house/clean': 1, 'face/noisy': 12, 'house/noisy': 11}

In [9]:
epochs.event_id

{'face/clean': 2, 'house/clean': 1, 'face/noisy': 12, 'house/noisy': 11}